In [1]:
import pandas as pd
from sklearn import cross_validation,metrics
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Загружаем данные из файлов 'angle','StructuralSums',etc
# Примечание: все эти файлы должны лежать в одной директории с модулем, либо
# нужно прописать путь к ним отдельно. 
def load_data(Balint = False, dataset=None): 
    angle = pd.read_csv('../FPs/angle', sep=',', header=None)
    SS = pd.read_csv('../FPs/StructuralSums', header=None)
    real = pd.read_csv('../FPs/real', header=None)
    volume = pd.read_csv('../FPs/volume', header = None)
    order = pd.read_csv('../FPs/order', header=None)
    E = pd.read_csv('../FPs/Energy', header=None)
    E_Balint = pd.read_csv('../FPs/Energy_Balint', header=None)
    poscar = pd.read_csv('../FPs/Poscars', header = None)
    
    E_Balint = E_Balint.drop(0, axis=1) # Далее - чутка подготовки данных, чтобы проще было работать с датафреймом
    
    name_list = ['angle', 'SS', 'real', 'volume', 'order']
    data_list = [angle, SS, real, volume, order]
    
    for i in range(len(data_list)):
        index = []
        for j in range(data_list[i].shape[1]):
            index.append(name_list[i] + '%i'% j)
        data_list[i].set_axis(1,[index])
    
    coefs = []
    
    for i in range(len(poscar)):
        if poscar[0][i] == 'Au':
            coefs.append(poscar[0][i+1])
    coefs = pd.DataFrame(coefs)
    for i in range(len(E)):
        E[0][i] = E[0][i]/float(coefs[0][i]) # Делим энергии на количество атомов
        E_Balint[1][i] = E_Balint[1][i]/float(coefs[0][i])
    FP = pd.concat([angle, SS, real, volume, order], axis=1) # Подготавливаем первичную матрицу Х
    
    
    index = [] #Чистим структуры с энергией больше нуля
    for i in range(len(E[0])):
        if E[0][i] > 0:
            index.append(i)
    E = E.drop(index)
    E_Balint = E_Balint.drop(index)
    FP = FP.drop(index)
    
    
    if dataset == 'old_FP': # Указатели на тип загружаемого датасета и энергий
        X = FP.astype(np.float32)
        print 'Old FP used'
    if dataset == 'new_FP':
        X = pd.read_csv('FP', sep=',', header = None, dtype=np.float32)
        X = X.drop(index)
        print 'New FP used'
    if not Balint: 
        y = E[0].astype(np.float32)
        print "DFT loaded"
    else:
        y = E_Balint[1].astype(np.float32)
        print "Balint loaded"
    return X,y # Итоговые данные

# Тут тоже немного чистим фичи, проверяем на отсутвие структур с энергией больше нуля
def clean_data(X,y,yb): 
    for i in yb:
        if i>0:
            X.drop(yb[yb==i].index[0], inplace=True)
            y.drop(yb[yb==i].index[0], inplace=True)
            yb.drop(yb[yb==i].index[0], inplace=True)
    X.index = range(0,len(X))
    y.index = range(0,len(y))
    yb.index = range(0,len(yb))
    print "All >0 deleted"
# Основная функция номер раз: подгружаем нашу загрузку данных и разбиваем на трейн и тест
# Параметры None отвечают стандартной загрузке датасета. Если нужно обучиться на других данных, то
# указываем их в аргументах
def get_train_test(X=None,y=None,Xb=None,yb=None,test_size=0.25):
    if (X == None) & (y==None):
        X,y = load_data(dataset='old_FP', Balint=False)
    print X.shape, y.shape
    if (Xb == None) & (yb==None):
        Xb, yb = load_data(dataset='old_FP', Balint=True)
    print Xb.shape, yb.shape
    clean_data(X,y,yb)
    # Разбиение на трейн и тест
    for itr, ite in cross_validation.ShuffleSplit(len(y), n_iter=5, train_size=(1.-test_size), test_size=test_size):
        pass
    
    xtrain = X.loc[itr]
    xtest = X.loc[ite]

    ytrain = y.loc[itr]
    ytest = y.loc[ite]

    ybtrain = yb.loc[itr]
    ybtest = yb.loc[ite]
    
    print 'Preparing delta(Y) as Y(DFT) - Y(Balint)'
    dytrain = ytrain - ybtrain
    dytest = ytest - ybtest
    return xtrain, xtest, dytrain,dytest
# Основная функция номер два. Идем по порядку.
# Описание параметров: max_depth - максимальная глубина базовых деревьев, eta - коэффициент, с
# которым новые деревья добавляются в композицию, numrounds - количество алгоритмов в композиции.
# Остальные параметры можно не трогать, но если очень хочется - то гуглим, что они делают, здесь
# кратко пробегусь. И еще, test_mode - флаг на отладку алгоритма при изменении параметров. Если хотим 
# оценить качество на кросс-валидации, то ставим True и смотрим. Нашли хорошие параметры - меняем
# обратно на False и запускаем алгоритм для предсказания энергий.
def xgboost_predict(xtrain,xtest,ytrain,ytest, max_depth=100,eta=0.01,numrounds=1000, test_mode = False):
    # Это словарь параметров, используемых алгоритмом
    param = {}
    param['max_depth'] = max_depth 
    param['subsample'] = 0.75 # Размер случайной подвыборки, на которой алгоритм обучается (уменьшает переобученность)
    param['rate_drop'] = 0.5 # Процент данных, которые пропускаются при обученнии (уменьшает переобученность)
    param['booster'] = 'gbtree'# Базовый алгортим - дерево
    param['objective'] = 'reg:linear' # Задача регрессии
    param['eval_metric'] = 'rmse' # Метрика качества
    param['eta'] = eta
    
    Xdatatrain = xgb.DMatrix(data = xtrain, label = ytrain) # Спец.вид матрицы, используемых алгоритмом
    plst = list(param.items()) 
    if test_mode == True:
        print param
        res = xgb.cv(params=param, dtrain=Xdatatrain, num_boost_round=numrounds, nfold=5)
    else:
        print param
        print 'Waiting for training...'
        bst = xgb.train(plst, Xdatatrain, numrounds)
        ypredxgb = bst.predict(xgb.DMatrix(xtest))
        print 'RMSE error:', np.sqrt(metrics.mean_squared_error(ytest, ypredxgb))
        return ypredxgb
print 'Use get_train_test() to load data, keep defaults unless you want to use your own data\nUse xgboost_predict() to run model'

Use get_train_test() to load data, keep defaults unless you want to use your own data
Use xgboost_predict() to run model
